In [12]:
import Robogame as rg
import networkx as nx
import altair as alt
import time, json
import pandas as pd
import numpy as np

# Simulator
This is an example of a simulator. It will create bets for the two players and the just see what happens.

I'm assuming two teams... one with the secret "alice" and one with "bob." 

To launch the server for this you can do the following in your server directory:

```python api.py -d ./example1 -t1s bob -t2s alice --matchsave bobvalice.json examplematch1```

This will launch the example1match from the example1 directory, using the two secrets (alice and bob). The log of the game will get saved to bobvalice.json which might be useful for analysis after the fact.

In [13]:
# let's read the real match data... this will give us the *real* answers
# you can modify this for different example files but remember to also launch the right server version
robotdata = pd.read_csv("../server/example1/examplematch1.robotdata.csv")

In [14]:
# let's create objects for our two teams
team1 = rg.Robogame("bob")
team2 = rg.Robogame("alice")

In [15]:
# tell the server we're ready
print(team1.setReady())
print(team2.setReady())

{'Result': 'OK'}
{'Result': 'OK'}


In [16]:
# wait for the match to launch
while(True):
    gametime = team1.getGameTime()
    timetogo = gametime['gamestarttime_secs'] - gametime['servertime_secs']
    if ('Error' in gametime):
        print("Error"+str(gametime))
        break
    if (timetogo <= 0):
        print("Let's go!")
        break
    
    print("waiting to launch... game will start in " + str(int(timetogo)))
    time.sleep(1) # sleep 1 second at a time, wait for the game to start

waiting to launch... game will start in 9
waiting to launch... game will start in 8
waiting to launch... game will start in 7
waiting to launch... game will start in 6
waiting to launch... game will start in 5
waiting to launch... game will start in 4
waiting to launch... game will start in 3
waiting to launch... game will start in 2
waiting to launch... game will start in 1
waiting to launch... game will start in 0
Let's go!


In [17]:
#let's try to figure out some strategy for each robot based on the real data

omniplayerAllBets1 = {}
omniplayerAllBets2 = {}


# grab the robots that we can guess for
for row in robotdata[robotdata.id < 100].sort_values('id').iterrows():
    row = row[1]                # get the row data
    expires = int(row.expires)  # figure out when the robot expires
    col = "t_"+str(expires)     # look up the value at that time
    valAtExp = row[col]         # this is the true value

    # you can also do things like only get for robots with a positive Productivity
    #if (row.Productivity > 0):
    #    .... robot is productive, make a guess
    #else:
    #    .... robot is unproductive, guess -1
    
    # pick a random number around the true answer (normal distrib with mean of valAtExp, stdev of 20)
    val = int(np.random.normal(valAtExp,20))
    if (val <  0):
        val = 0
    if (val > 100):
        val = 100
        
    omniplayerAllBets1[row.id] = val
    
    # pick a random number around the true answer (normal distrib with mean of valAtExp, stdev of 10)
    # this will be for team2 which are going to be slightly more accurate
    val = int(np.random.normal(valAtExp,10))
    if (val <  0):
        val = 0
    if (val > 100):
        val = 100
        
    omniplayerAllBets2[row.id] = val


In [18]:
# set the bets for the two teams
team1.setBets(omniplayerAllBets1)
team2.setBets(omniplayerAllBets2)

{'Result': 'OK'}

In [19]:
# sit and wait until the 10 minutes are done
# this will show you the scores as the game advances
for i in np.arange(0,100):
    x = team1.getRobotInfo()
    t = team1.getGameTime()
    if 'curtime' in t:
        if (t['curtime'] >= 100):
            break
    else:
        break
    
    # we're going to ask for hints even though we won't use them here
    team1.getHints()
    team2.getHints()
    
    print("time: ",t['curtime'])
    print(x[['winner','Productivity']].groupby(['winner']).sum())
    time.sleep(6)

time:  0.13
        Productivity
winner              
-2               0.0
time:  1.14
        Productivity
winner              
-2               0.0
time:  2.15
        Productivity
winner              
-2               0.0
time:  3.16
        Productivity
winner              
-2               0.0
time:  4.17
        Productivity
winner              
-2               0.0
time:  5.18
        Productivity
winner              
-2               0.0
time:  6.19
        Productivity
winner              
-2               0.0
time:  7.2
        Productivity
winner              
-2               0.0
time:  8.21
        Productivity
winner              
-2               0.0
time:  9.22
        Productivity
winner              
-2               0.0
time:  10.23
        Productivity
winner              
-2               0.0
time:  11.24
        Productivity
winner              
-2               0.0
time:  12.25
        Productivity
winner              
-2               0.0
time:  13.26
        Pr

time:  79.38
        Productivity
winner              
-2          0.000000
 1       1358.510413
 2       2258.679969
time:  80.41
        Productivity
winner              
-2          0.000000
 1       1358.510413
 2       2330.940754
time:  81.43
        Productivity
winner              
-2          0.000000
 1       1358.510413
 2       2430.940754
time:  82.45
        Productivity
winner              
-2          0.000000
 1       1358.510413
 2       2516.210926
time:  83.47
        Productivity
winner              
-2          0.000000
 1       1358.510413
 2       2516.210926
time:  84.49
        Productivity
winner              
-2          0.000000
 1       1397.082953
 2       2516.210926
time:  85.52
        Productivity
winner              
-2          0.000000
 1       1397.082953
 2       2516.210926
time:  86.54
        Productivity
winner              
-2          0.000000
 1       1397.082953
 2       2516.210926
time:  87.56
        Productivity
winner              
-

In [ ]:
# that's more or less it... you can modify the guesses us different strategies
# for example, you can also consider the network